In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 64
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.20648193359375 | KNN Loss: 6.229707717895508 | BCE Loss: 1.9767743349075317
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.153791427612305 | KNN Loss: 6.229312896728516 | BCE Loss: 1.9244780540466309
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.150104522705078 | KNN Loss: 6.22951078414917 | BCE Loss: 1.9205937385559082
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.113521575927734 | KNN Loss: 6.229531288146973 | BCE Loss: 1.8839905261993408
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.16419506072998 | KNN Loss: 6.228993892669678 | BCE Loss: 1.9352010488510132
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.130905151367188 | KNN Loss: 6.228990077972412 | BCE Loss: 1.9019153118133545
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.154356956481934 | KNN Loss: 6.228734970092773 | BCE Loss: 1.9256222248077393
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.132513046264648 | KNN Loss: 6.228709697723389 | BCE Loss: 1.90380287

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.164376258850098 | KNN Loss: 5.898101329803467 | BCE Loss: 1.2662748098373413
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.144925117492676 | KNN Loss: 5.868739604949951 | BCE Loss: 1.2761855125427246
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.037556171417236 | KNN Loss: 5.836838245391846 | BCE Loss: 1.2007179260253906
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.977527618408203 | KNN Loss: 5.7611799240112305 | BCE Loss: 1.2163479328155518
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.82755184173584 | KNN Loss: 5.637224197387695 | BCE Loss: 1.1903276443481445
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.773477554321289 | KNN Loss: 5.586361885070801 | BCE Loss: 1.1871156692504883
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 6.676501274108887 | KNN Loss: 5.483994007110596 | BCE Loss: 1.192507028579712
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 6.55367374420166 | KNN Loss: 5.393153667449951 | BCE Loss: 

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 5.5549540519714355 | KNN Loss: 4.488445281982422 | BCE Loss: 1.0665087699890137
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 5.548538684844971 | KNN Loss: 4.491568088531494 | BCE Loss: 1.0569705963134766
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 5.554873466491699 | KNN Loss: 4.491104602813721 | BCE Loss: 1.0637686252593994
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 5.548341751098633 | KNN Loss: 4.495208740234375 | BCE Loss: 1.053133249282837
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 5.5611186027526855 | KNN Loss: 4.513321399688721 | BCE Loss: 1.0477972030639648
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 5.552997589111328 | KNN Loss: 4.482912063598633 | BCE Loss: 1.0700852870941162
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 5.53004264831543 | KNN Loss: 4.47650146484375 | BCE Loss: 1.0535414218902588
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 5.532318592071533 | KNN Loss: 4.472842216491699 | BCE Loss

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 5.465092658996582 | KNN Loss: 4.412795543670654 | BCE Loss: 1.0522971153259277
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 5.471210479736328 | KNN Loss: 4.431530475616455 | BCE Loss: 1.039679765701294
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 5.5225958824157715 | KNN Loss: 4.42623233795166 | BCE Loss: 1.0963634252548218
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 5.483162879943848 | KNN Loss: 4.425594329833984 | BCE Loss: 1.0575687885284424
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 5.465487480163574 | KNN Loss: 4.40500020980835 | BCE Loss: 1.0604875087738037
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 5.452884197235107 | KNN Loss: 4.401498317718506 | BCE Loss: 1.0513858795166016
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 5.537181854248047 | KNN Loss: 4.447823524475098 | BCE Loss: 1.0893582105636597
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 5.439001083374023 | KNN Loss: 4.420112609863281 | BCE Loss:

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 5.449646949768066 | KNN Loss: 4.367004871368408 | BCE Loss: 1.0826420783996582
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 5.438197135925293 | KNN Loss: 4.406385898590088 | BCE Loss: 1.031810998916626
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 5.438326358795166 | KNN Loss: 4.400182723999023 | BCE Loss: 1.038143515586853
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 5.430689811706543 | KNN Loss: 4.380826473236084 | BCE Loss: 1.0498634576797485
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 5.429266929626465 | KNN Loss: 4.39487361907959 | BCE Loss: 1.034393072128296
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 5.439969062805176 | KNN Loss: 4.393905162811279 | BCE Loss: 1.0460641384124756
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 5.422880172729492 | KNN Loss: 4.380152225494385 | BCE Loss: 1.0427279472351074
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 5.443521499633789 | KNN Loss: 4.396193027496338 | BCE Loss: 1

...

Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 5.423148155212402 | KNN Loss: 4.35068416595459 | BCE Loss: 1.0724639892578125
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 5.461158275604248 | KNN Loss: 4.419189453125 | BCE Loss: 1.0419687032699585
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 5.389246463775635 | KNN Loss: 4.370909690856934 | BCE Loss: 1.0183367729187012
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 5.395580768585205 | KNN Loss: 4.372767448425293 | BCE Loss: 1.0228132009506226
Epoch 450 / 500 | iteration 0 / 30 | Total Loss: 5.377960205078125 | KNN Loss: 4.34993839263916 | BCE Loss: 1.0280218124389648
Epoch 450 / 500 | iteration 5 / 30 | Total Loss: 5.393111228942871 | KNN Loss: 4.352309703826904 | BCE Loss: 1.0408015251159668
Epoch 450 / 500 | iteration 10 / 30 | Total Loss: 5.331720352172852 | KNN Loss: 4.331087589263916 | BCE Loss: 1.0006327629089355
Epoch 450 / 500 | iteration 15 / 30 | Total Loss: 5.380775451660156 | KNN Loss: 4.344493389129639 | BCE 

Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 5.386999607086182 | KNN Loss: 4.380929946899414 | BCE Loss: 1.0060697793960571
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 5.334714889526367 | KNN Loss: 4.324290752410889 | BCE Loss: 1.0104238986968994
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 5.385002613067627 | KNN Loss: 4.346275806427002 | BCE Loss: 1.0387266874313354
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 5.4249653816223145 | KNN Loss: 4.3769330978393555 | BCE Loss: 1.0480321645736694
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 5.290914535522461 | KNN Loss: 4.295884132385254 | BCE Loss: 0.9950305819511414
Epoch 460 / 500 | iteration 25 / 30 | Total Loss: 5.386981010437012 | KNN Loss: 4.3875532150268555 | BCE Loss: 0.9994277954101562
Epoch 461 / 500 | iteration 0 / 30 | Total Loss: 5.370757579803467 | KNN Loss: 4.334097385406494 | BCE Loss: 1.0366601943969727
Epoch 461 / 500 | iteration 5 / 30 | Total Loss: 5.3638505935668945 | KNN Loss: 4.335745811462402

Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 5.423464298248291 | KNN Loss: 4.391591548919678 | BCE Loss: 1.0318726301193237
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 5.368599891662598 | KNN Loss: 4.357300281524658 | BCE Loss: 1.01129949092865
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 5.375969886779785 | KNN Loss: 4.3642730712890625 | BCE Loss: 1.0116968154907227
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 5.359688758850098 | KNN Loss: 4.33807897567749 | BCE Loss: 1.0216097831726074
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 5.340062141418457 | KNN Loss: 4.32853889465332 | BCE Loss: 1.0115234851837158
Epoch 471 / 500 | iteration 15 / 30 | Total Loss: 5.390985488891602 | KNN Loss: 4.370636463165283 | BCE Loss: 1.0203490257263184
Epoch 471 / 500 | iteration 20 / 30 | Total Loss: 5.374786853790283 | KNN Loss: 4.332377910614014 | BCE Loss: 1.0424089431762695
Epoch 471 / 500 | iteration 25 / 30 | Total Loss: 5.377124309539795 | KNN Loss: 4.357393741607666 | BC

Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 5.449256420135498 | KNN Loss: 4.416590213775635 | BCE Loss: 1.0326663255691528
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 5.333684921264648 | KNN Loss: 4.340879917144775 | BCE Loss: 0.9928052425384521
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 5.391486167907715 | KNN Loss: 4.374935150146484 | BCE Loss: 1.0165507793426514
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 5.351450443267822 | KNN Loss: 4.331624984741211 | BCE Loss: 1.0198253393173218
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 5.394283294677734 | KNN Loss: 4.3771209716796875 | BCE Loss: 1.0171622037887573
Epoch 482 / 500 | iteration 5 / 30 | Total Loss: 5.38389778137207 | KNN Loss: 4.361886024475098 | BCE Loss: 1.0220119953155518
Epoch 482 / 500 | iteration 10 / 30 | Total Loss: 5.368283271789551 | KNN Loss: 4.354225158691406 | BCE Loss: 1.0140578746795654
Epoch 482 / 500 | iteration 15 / 30 | Total Loss: 5.428168296813965 | KNN Loss: 4.3803253173828125 

Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 5.382512092590332 | KNN Loss: 4.365140438079834 | BCE Loss: 1.0173718929290771
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 5.305492401123047 | KNN Loss: 4.301912784576416 | BCE Loss: 1.0035796165466309
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 5.386650085449219 | KNN Loss: 4.372052192687988 | BCE Loss: 1.0145981311798096
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 5.3594818115234375 | KNN Loss: 4.342615127563477 | BCE Loss: 1.016866683959961
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 5.398952960968018 | KNN Loss: 4.367936611175537 | BCE Loss: 1.0310163497924805
Epoch 492 / 500 | iteration 25 / 30 | Total Loss: 5.428266525268555 | KNN Loss: 4.396023750305176 | BCE Loss: 1.032242774963379
Epoch 493 / 500 | iteration 0 / 30 | Total Loss: 5.367700099945068 | KNN Loss: 4.355287075042725 | BCE Loss: 1.0124129056930542
Epoch 493 / 500 | iteration 5 / 30 | Total Loss: 5.358769416809082 | KNN Loss: 4.3501362800598145 | B

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.7441,  2.6732,  3.0950,  4.1396,  4.0467,  0.6395,  1.9095,  2.7127,
          1.7671,  1.6543,  2.7099,  1.4446,  1.1443,  2.1694,  1.2966,  2.0321,
          2.4437,  1.9823,  1.9205,  2.8463,  1.7464,  3.4970,  1.5933,  3.1552,
          2.5494,  1.8534,  1.7290,  1.4775,  1.4493,  0.4399,  0.1224,  1.0770,
          0.2949,  1.1134,  1.3717,  1.1682,  1.3365,  3.8645,  0.7598,  1.3859,
          1.3416, -0.6291, -0.3325,  2.8607,  2.1404,  0.6125, -0.1257, -0.0143,
          1.4580,  2.1000,  2.1628,  0.0753,  1.7593,  0.5025, -0.3609,  1.1956,
          1.8183,  1.4060,  1.6863,  1.7672,  0.6736,  0.8092,  0.1594,  1.3707,
          1.1880,  1.9930, -1.7575,  0.3536,  1.8959,  1.8832,  3.0835,  0.2219,
          1.3353,  1.6725,  1.6955,  1.6607,  0.1663,  0.6747,  0.0909,  1.3627,
         -0.0319,  0.7510,  2.2128, -0.3403,  0.3755, -1.0097, -2.2813, -0.5493,
          0.6126, -1.6691,  0.3744, -0.0092, -0.4956, -0.7589,  0.5502,  1.3111,
         -0.6540, -1.2032,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].to('cpu') for d in dataset]

In [11]:
model = model.eval().to('cpu')
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 92.22it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.1, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [17]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [18]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [19]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [20]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [21]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [22]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [23]:
tensor_dataset = torch.stack(dataset_)

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 025 | Total loss: 9.590 | Reg loss: 0.007 | Tree loss: 9.590 | Accuracy: 0.000000 | 0.097 sec/iter
Epoch: 00 | Batch: 001 / 025 | Total loss: 9.587 | Reg loss: 0.007 | Tree loss: 9.587 | Accuracy: 0.000000 | 0.086 sec/iter
Epoch: 00 | Batch: 002 / 025 | Total loss: 9.571 | Reg loss: 0.007 | Tree loss: 9.571 | Accuracy: 0.000000 | 0.079 sec/iter
Epoch: 00 | Batch: 003 / 025 | Total loss: 9.558 | Reg loss: 0.007 | Tree loss: 9.558 | Accuracy: 0.000000 | 0.075 sec/iter
Epoch: 00 | Batch: 004 / 025 | Total loss: 9.561 | Reg loss: 0.006 | Tree loss: 9.561 | Accuracy: 0.000000 | 0.072 sec/iter
Epoch: 00 | Batch: 005 / 025 | Total loss: 9.550 | Reg loss: 0.006 | Tree loss: 9.550 | Accuracy: 0.000000 | 0.07 sec/iter
Epoch: 00 | Batch: 006 / 025 | Total loss: 9.535 | Reg loss: 0.006 | Tree loss: 9.535 | Accuracy: 0.000000 | 0.068 sec/iter
Epoch: 00 | Batch: 007 / 025 | Total loss: 9

Epoch: 02 | Batch: 015 / 025 | Total loss: 9.186 | Reg loss: 0.008 | Tree loss: 9.186 | Accuracy: 0.101562 | 0.069 sec/iter
Epoch: 02 | Batch: 016 / 025 | Total loss: 9.175 | Reg loss: 0.008 | Tree loss: 9.175 | Accuracy: 0.111328 | 0.069 sec/iter
Epoch: 02 | Batch: 017 / 025 | Total loss: 9.168 | Reg loss: 0.008 | Tree loss: 9.168 | Accuracy: 0.093750 | 0.069 sec/iter
Epoch: 02 | Batch: 018 / 025 | Total loss: 9.159 | Reg loss: 0.009 | Tree loss: 9.159 | Accuracy: 0.109375 | 0.069 sec/iter
Epoch: 02 | Batch: 019 / 025 | Total loss: 9.142 | Reg loss: 0.009 | Tree loss: 9.142 | Accuracy: 0.109375 | 0.069 sec/iter
Epoch: 02 | Batch: 020 / 025 | Total loss: 9.141 | Reg loss: 0.009 | Tree loss: 9.141 | Accuracy: 0.099609 | 0.069 sec/iter
Epoch: 02 | Batch: 021 / 025 | Total loss: 9.129 | Reg loss: 0.010 | Tree loss: 9.129 | Accuracy: 0.089844 | 0.069 sec/iter
Epoch: 02 | Batch: 022 / 025 | Total loss: 9.128 | Reg loss: 0.010 | Tree loss: 9.128 | Accuracy: 0.115234 | 0.069 sec/iter
Epoch: 0

Epoch: 05 | Batch: 003 / 025 | Total loss: 8.896 | Reg loss: 0.010 | Tree loss: 8.896 | Accuracy: 0.117188 | 0.07 sec/iter
Epoch: 05 | Batch: 004 / 025 | Total loss: 8.892 | Reg loss: 0.010 | Tree loss: 8.892 | Accuracy: 0.078125 | 0.07 sec/iter
Epoch: 05 | Batch: 005 / 025 | Total loss: 8.877 | Reg loss: 0.010 | Tree loss: 8.877 | Accuracy: 0.111328 | 0.07 sec/iter
Epoch: 05 | Batch: 006 / 025 | Total loss: 8.872 | Reg loss: 0.010 | Tree loss: 8.872 | Accuracy: 0.111328 | 0.07 sec/iter
Epoch: 05 | Batch: 007 / 025 | Total loss: 8.862 | Reg loss: 0.010 | Tree loss: 8.862 | Accuracy: 0.105469 | 0.07 sec/iter
Epoch: 05 | Batch: 008 / 025 | Total loss: 8.851 | Reg loss: 0.011 | Tree loss: 8.851 | Accuracy: 0.130859 | 0.07 sec/iter
Epoch: 05 | Batch: 009 / 025 | Total loss: 8.839 | Reg loss: 0.011 | Tree loss: 8.839 | Accuracy: 0.107422 | 0.07 sec/iter
Epoch: 05 | Batch: 010 / 025 | Total loss: 8.832 | Reg loss: 0.011 | Tree loss: 8.832 | Accuracy: 0.101562 | 0.07 sec/iter
Epoch: 05 | Batc

Epoch: 07 | Batch: 017 / 025 | Total loss: 8.425 | Reg loss: 0.017 | Tree loss: 8.425 | Accuracy: 0.101562 | 0.069 sec/iter
Epoch: 07 | Batch: 018 / 025 | Total loss: 8.410 | Reg loss: 0.017 | Tree loss: 8.410 | Accuracy: 0.119141 | 0.069 sec/iter
Epoch: 07 | Batch: 019 / 025 | Total loss: 8.395 | Reg loss: 0.017 | Tree loss: 8.395 | Accuracy: 0.115234 | 0.069 sec/iter
Epoch: 07 | Batch: 020 / 025 | Total loss: 8.386 | Reg loss: 0.018 | Tree loss: 8.386 | Accuracy: 0.093750 | 0.069 sec/iter
Epoch: 07 | Batch: 021 / 025 | Total loss: 8.383 | Reg loss: 0.018 | Tree loss: 8.383 | Accuracy: 0.076172 | 0.069 sec/iter
Epoch: 07 | Batch: 022 / 025 | Total loss: 8.365 | Reg loss: 0.018 | Tree loss: 8.365 | Accuracy: 0.099609 | 0.069 sec/iter
Epoch: 07 | Batch: 023 / 025 | Total loss: 8.344 | Reg loss: 0.019 | Tree loss: 8.344 | Accuracy: 0.107422 | 0.069 sec/iter
Epoch: 07 | Batch: 024 / 025 | Total loss: 8.332 | Reg loss: 0.019 | Tree loss: 8.332 | Accuracy: 0.103125 | 0.069 sec/iter
Average 

Epoch: 10 | Batch: 007 / 025 | Total loss: 8.009 | Reg loss: 0.020 | Tree loss: 8.009 | Accuracy: 0.121094 | 0.068 sec/iter
Epoch: 10 | Batch: 008 / 025 | Total loss: 8.001 | Reg loss: 0.020 | Tree loss: 8.001 | Accuracy: 0.109375 | 0.068 sec/iter
Epoch: 10 | Batch: 009 / 025 | Total loss: 7.980 | Reg loss: 0.020 | Tree loss: 7.980 | Accuracy: 0.123047 | 0.068 sec/iter
Epoch: 10 | Batch: 010 / 025 | Total loss: 7.955 | Reg loss: 0.020 | Tree loss: 7.955 | Accuracy: 0.113281 | 0.068 sec/iter
Epoch: 10 | Batch: 011 / 025 | Total loss: 7.922 | Reg loss: 0.021 | Tree loss: 7.922 | Accuracy: 0.101562 | 0.068 sec/iter
Epoch: 10 | Batch: 012 / 025 | Total loss: 7.910 | Reg loss: 0.021 | Tree loss: 7.910 | Accuracy: 0.121094 | 0.068 sec/iter
Epoch: 10 | Batch: 013 / 025 | Total loss: 7.901 | Reg loss: 0.021 | Tree loss: 7.901 | Accuracy: 0.099609 | 0.068 sec/iter
Epoch: 10 | Batch: 014 / 025 | Total loss: 7.861 | Reg loss: 0.021 | Tree loss: 7.861 | Accuracy: 0.101562 | 0.068 sec/iter
Epoch: 1

...

Epoch: 87 | Batch: 016 / 025 | Total loss: 2.355 | Reg loss: 0.029 | Tree loss: 2.355 | Accuracy: 0.175781 | 0.071 sec/iter
Epoch: 87 | Batch: 017 / 025 | Total loss: 2.334 | Reg loss: 0.029 | Tree loss: 2.334 | Accuracy: 0.160156 | 0.071 sec/iter
Epoch: 87 | Batch: 018 / 025 | Total loss: 2.317 | Reg loss: 0.029 | Tree loss: 2.317 | Accuracy: 0.173828 | 0.071 sec/iter
Epoch: 87 | Batch: 019 / 025 | Total loss: 2.341 | Reg loss: 0.029 | Tree loss: 2.341 | Accuracy: 0.152344 | 0.071 sec/iter
Epoch: 87 | Batch: 020 / 025 | Total loss: 2.353 | Reg loss: 0.029 | Tree loss: 2.353 | Accuracy: 0.177734 | 0.071 sec/iter
Epoch: 87 | Batch: 021 / 025 | Total loss: 2.351 | Reg loss: 0.029 | Tree loss: 2.351 | Accuracy: 0.162109 | 0.071 sec/iter
Epoch: 87 | Batch: 022 / 025 | Total loss: 2.337 | Reg loss: 0.029 | Tree loss: 2.337 | Accuracy: 0.150391 | 0.071 sec/iter
Epoch: 87 | Batch: 023 / 025 | Total loss: 2.287 | Reg loss: 0.029 | Tree loss: 2.287 | Accuracy: 0.214844 | 0.071 sec/iter
Epoch: 8

Epoch: 90 | Batch: 003 / 025 | Total loss: 2.386 | Reg loss: 0.029 | Tree loss: 2.386 | Accuracy: 0.175781 | 0.071 sec/iter
Epoch: 90 | Batch: 004 / 025 | Total loss: 2.413 | Reg loss: 0.029 | Tree loss: 2.413 | Accuracy: 0.177734 | 0.071 sec/iter
Epoch: 90 | Batch: 005 / 025 | Total loss: 2.438 | Reg loss: 0.029 | Tree loss: 2.438 | Accuracy: 0.140625 | 0.071 sec/iter
Epoch: 90 | Batch: 006 / 025 | Total loss: 2.374 | Reg loss: 0.029 | Tree loss: 2.374 | Accuracy: 0.167969 | 0.071 sec/iter
Epoch: 90 | Batch: 007 / 025 | Total loss: 2.356 | Reg loss: 0.029 | Tree loss: 2.356 | Accuracy: 0.175781 | 0.071 sec/iter
Epoch: 90 | Batch: 008 / 025 | Total loss: 2.390 | Reg loss: 0.029 | Tree loss: 2.390 | Accuracy: 0.158203 | 0.071 sec/iter
Epoch: 90 | Batch: 009 / 025 | Total loss: 2.401 | Reg loss: 0.029 | Tree loss: 2.401 | Accuracy: 0.175781 | 0.071 sec/iter
Epoch: 90 | Batch: 010 / 025 | Total loss: 2.379 | Reg loss: 0.029 | Tree loss: 2.379 | Accuracy: 0.173828 | 0.071 sec/iter
Epoch: 9

Epoch: 92 | Batch: 019 / 025 | Total loss: 2.336 | Reg loss: 0.029 | Tree loss: 2.336 | Accuracy: 0.146484 | 0.071 sec/iter
Epoch: 92 | Batch: 020 / 025 | Total loss: 2.363 | Reg loss: 0.029 | Tree loss: 2.363 | Accuracy: 0.152344 | 0.071 sec/iter
Epoch: 92 | Batch: 021 / 025 | Total loss: 2.297 | Reg loss: 0.029 | Tree loss: 2.297 | Accuracy: 0.148438 | 0.071 sec/iter
Epoch: 92 | Batch: 022 / 025 | Total loss: 2.311 | Reg loss: 0.029 | Tree loss: 2.311 | Accuracy: 0.183594 | 0.071 sec/iter
Epoch: 92 | Batch: 023 / 025 | Total loss: 2.303 | Reg loss: 0.029 | Tree loss: 2.303 | Accuracy: 0.150391 | 0.071 sec/iter
Epoch: 92 | Batch: 024 / 025 | Total loss: 2.285 | Reg loss: 0.029 | Tree loss: 2.285 | Accuracy: 0.193750 | 0.071 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 93 | Batch: 000 / 025 | Total loss: 2.425 | Reg loss: 0.029 | Tree los

Epoch: 95 | Batch: 007 / 025 | Total loss: 2.368 | Reg loss: 0.029 | Tree loss: 2.368 | Accuracy: 0.189453 | 0.072 sec/iter
Epoch: 95 | Batch: 008 / 025 | Total loss: 2.404 | Reg loss: 0.029 | Tree loss: 2.404 | Accuracy: 0.158203 | 0.072 sec/iter
Epoch: 95 | Batch: 009 / 025 | Total loss: 2.338 | Reg loss: 0.029 | Tree loss: 2.338 | Accuracy: 0.193359 | 0.072 sec/iter
Epoch: 95 | Batch: 010 / 025 | Total loss: 2.348 | Reg loss: 0.029 | Tree loss: 2.348 | Accuracy: 0.199219 | 0.072 sec/iter
Epoch: 95 | Batch: 011 / 025 | Total loss: 2.421 | Reg loss: 0.029 | Tree loss: 2.421 | Accuracy: 0.160156 | 0.072 sec/iter
Epoch: 95 | Batch: 012 / 025 | Total loss: 2.354 | Reg loss: 0.029 | Tree loss: 2.354 | Accuracy: 0.158203 | 0.072 sec/iter
Epoch: 95 | Batch: 013 / 025 | Total loss: 2.334 | Reg loss: 0.029 | Tree loss: 2.334 | Accuracy: 0.160156 | 0.072 sec/iter
Epoch: 95 | Batch: 014 / 025 | Total loss: 2.305 | Reg loss: 0.029 | Tree loss: 2.305 | Accuracy: 0.185547 | 0.072 sec/iter
Epoch: 9

Epoch: 97 | Batch: 021 / 025 | Total loss: 2.277 | Reg loss: 0.029 | Tree loss: 2.277 | Accuracy: 0.177734 | 0.072 sec/iter
Epoch: 97 | Batch: 022 / 025 | Total loss: 2.266 | Reg loss: 0.029 | Tree loss: 2.266 | Accuracy: 0.189453 | 0.072 sec/iter
Epoch: 97 | Batch: 023 / 025 | Total loss: 2.287 | Reg loss: 0.029 | Tree loss: 2.287 | Accuracy: 0.152344 | 0.072 sec/iter
Epoch: 97 | Batch: 024 / 025 | Total loss: 2.272 | Reg loss: 0.029 | Tree loss: 2.272 | Accuracy: 0.181250 | 0.072 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 98 | Batch: 000 / 025 | Total loss: 2.456 | Reg loss: 0.028 | Tree loss: 2.456 | Accuracy: 0.164062 | 0.072 sec/iter
Epoch: 98 | Batch: 001 / 025 | Total loss: 2.457 | Reg loss: 0.028 | Tree loss: 2.457 | Accuracy: 0.160156 | 0.072 sec/iter
Epoch: 98 | Batch: 002 / 025 | Total loss: 2.412 | Reg loss: 0.028 | Tree los

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 4.333333333333333


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 12


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

11084
============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
1524
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
Average comprehensibility: 21.0
std comprehensibility: 7.14142842854285
var comprehensibility: 51.0
minimum comprehensibility: 10
maximum comprehensibility: 30


/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
